Here we are going to make the Vector Database Indexing with Chroma

We are going to create the embeddings of some pokemon characteristics that are into the JSON files that we created by using the Poke API for each Pokemons

Import dependencies

In [1]:
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
from dotenv import load_dotenv
import os
import json

import time

from langchain_community.embeddings import HuggingFaceEmbeddings

c:\Users\rayan\anaconda3\envs\pokemon-project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


At the beginning, we wanted to create chroma with Gemini but we used all our credits so we changed and we used HuggingFaceEmbeddings to do that.

In [1]:
#Load the Gemini API key from the .env file.
load_dotenv()

NameError: name 'load_dotenv' is not defined

In [ ]:
#Folder that contains JSON file for each pokemon
POKEDEX_DIRECTORY = "../data/pokedex"
#Folder that will contain the chroma embeddings 
CHROMA_DIRECTORY = "../data/chroma_pokedex"

Checks the existent of the chroma database

In [4]:
def chroma_exists():
    return(
        os.path.exists(CHROMA_DIRECTORY) and os.path.isfile(os.path.join(CHROMA_DIRECTORY, "index.sqlite3"))
    )

Function that loop through our JSON pokedex to convert the files in it into LangChain Documents 

In [ ]:
def build_documents():
    #Builds a list of LangChain Document objects from Pokédex JSON files.
    documents = []
    #Loop through all JSON files of the pokedex folder
    for filename in os.listdir(POKEDEX_DIRECTORY):
        # Open and load each JSON file with UTF-8 encoding
        with open(os.path.join(POKEDEX_DIRECTORY, filename), "r", encoding="utf-8") as f:
            data = json.load(f)
            # Format the Pokémon data into a structured text content string            
            content = f"""
            Name: {data['Name']}
            Types: {', '.join(data['Types'])}

            Description:
            {data['Description']}

            Weaknesses:
            {', '.join(data['Weaknesses'])}

            Resistances:
            {', '.join(data['Resistances'])}

            Immunities:
            {', '.join(data['Immunities'])}
            """.strip()

            # Create a Document object with the formatted content and metadata (name and source file)
            documents.append(Document(page_content=content, metadata={"pokemon": data['Name'], "filename": filename}))

    # Return the complete list of Document objects for vector embedding
    return documents



In [ ]:
def build_chroma(force: bool = False):
    # Check if the Chroma database already exists
    if chroma_exists() and not force:
        return

    # Initialize the embedding model using HuggingFace's sentence-transformers
    embeddings = HuggingFaceEmbeddings(
        model_name = "sentence-transformers/all-MiniLM-L6-v2"
    )

    # Build the list of LangChain Document objects from Pokédex JSON files
    documents = build_documents()

    # Create and persist the Chroma vector database from the documents
    chroma_db = Chroma.from_documents(
        documents = documents,  # Documents containing Pokémon information
        embedding = embeddings,  # The embedding model to use for vectorization
        persist_directory = CHROMA_DIRECTORY,  # Directory where the database will be saved
    )

Calling the function

In [7]:
build_chroma()

C:\Users\rayan\AppData\Local\Temp\ipykernel_7128\1038197454.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


C:\Users\rayan\AppData\Local\Temp\ipykernel_7128\1038197454.py:19: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  chroma_db.persist()
